# Synthetic Dataset Generation (SDG) using TAO StyleGAN-XL

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">


## What is StyleGAN-XL ?

StyleGAN-XL is a synthetic dataset generation model for us to generate more images for downstream task such as classification and segmentation. The techniques could be powerful especially when the existing real dataset is not sufficient for training downstream models.


### Sample generated images of the trained StyleGAN-XL using endoscopy images
<div style="padding-left: 100px;">
<img src="images/fakes011290.png" width="1500"/>
</div>

## Learning Objectives

In this notebook, you will learn how to use TAO to `train`, `evaluate`, and `inference` with StyleGAN-XL


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/bevfusion/results`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=/raid/path/to/local/tao-experiments

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "stylegan_xl", "data")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "stylegan_xl", "results")
os.environ["LOCAL_CACHE_DIR"] = os.path.join(os.environ['HOME'], ".cache")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/bevfusion

# The sample spec files are present in the same path as the downloaded samples.|
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tao_configs = {
   "Mounts":[
         # Mapping the Local project directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
       {
           "source": os.environ["LOCAL_CACHE_DIR"],
           "destination": "/.cache"
       },
   ],
   "Envs": [
        {
            "variable": "TAO_TOOLKIT_CACHE",
            "value": "/.cache",
        }
    ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         },
        "network": "host"
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tao_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

In [ ]:
!pip install pillow
!pip install torch
!pip install numpy
!pip install dill

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
# View the versions of the TAO launcher
!tao info
!tao model -h

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare Raw Dataset

We will use the https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data dataset to train a StyleGAN-XL model. This dataset consists of six classes of images, organized into a folder structure where each class corresponds to a subfolder containing its respective images. The hierarchy of the `neu-metal-surface-defects-data` dataset is as follows:

<pre style="font-family: 'Courier New', monospace; font-size: 14px; line-height: 1.5; color: #333;">
neu-metal-surface-defects-data/
├── class_1/
│   ├── class_1_image1.png
│   ├── class_1_image2.png
│   ├── ...
│   ├── class_1_imageN.png
├── class_2/
│   ├── class_2_image1.png
│   ├── class_2_image2.png
│   ├── ...
│   ├── class_2_imageN.png
</pre>


In [ ]:
#!/bin/bash
! curl -L -o $HOST_DATA_DIR/neu-metal-surface-defects-data.zip\
  https://www.kaggle.com/api/v1/datasets/download/fantacher/neu-metal-surface-defects-data

In [ ]:
import os
import zipfile
from PIL import Image
import shutil

# Path to the ZIP file
zip_file_path = os.path.join(os.environ["HOST_DATA_DIR"], "neu-metal-surface-defects-data.zip")

# Directory to extract files
extract_dir = os.environ["HOST_DATA_DIR"]

# Desired folder name
desired_folder_name = "NEU_Metal_Surface_Defects_Data"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Original extracted folder name
original_folder_name = os.path.join(extract_dir, "NEU Metal Surface Defects Data")

# Rename the folder
renamed_folder = os.path.join(extract_dir, desired_folder_name)
if os.path.exists(renamed_folder):
    shutil.rmtree(renamed_folder)  # Remove if the target folder already exists
os.rename(original_folder_name, renamed_folder)

# Convert .bmp files to .png
for root, dirs, files in os.walk(renamed_folder):
    for file in files:
        if file.endswith(".bmp"):
            bmp_file_path = os.path.join(root, file)
            png_file_path = os.path.join(root, file.replace(".bmp", ".png"))

            # Convert BMP to PNG
            with Image.open(bmp_file_path) as img:
                img.convert("RGB").save(png_file_path, "PNG")

            # Remove the original BMP file
            os.remove(bmp_file_path)

print(f"Extraction and conversion completed. Files are in: {renamed_folder}")

In [ ]:
! ls -all $HOST_DATA_DIR/NEU_Metal_Surface_Defects_Data

### 2.2 Preprocess dataset

We provide a dataset tool entrypoint, `dataset_convert`, to convert the dataset folder described above into a zipped file. This tool serves two main purposes:

1. **Dataset Portability and Performance**: Zipping datasets simplifies transferring them between file servers and clusters and may improve training performance when using network file systems.  
2. **Image Preprocessing**: StyleGAN-XL requires square, fixed-resolution images for training. The tool can crop and/or resize images to meet the resolution requirements of StyleGAN-XL's progressive training workflow. This training process involves starting with lower resolutions (e.g., `16x16`) and progressively increasing to higher resolutions (e.g., `256x256`). Consequently, we need multiple versions of the dataset, such as `16x16`, `32x32`, `64x64`, `128x128`, and `256x256`.

#### Sample Execution:  
To create a zipped dataset with `16x16` resolution images:  
```bash
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_16.zip \
                  resolution=[16,16]
```

By default, the tool resizes images to the target resolution, which may distort the images. To avoid distortion, you can enable the `center-crop` option before resizing by adding `transform=center-crop`. Note that while center-cropping preserves image proportions, it may crop out important border information, especially for images with aspect ratios far from 1:1.  

Example with center-cropping:  
```bash
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_16.zip \
                  resolution=[16,16] \
                  transform=center-crop
```

#### Output Structure:  
The zipped file reorganizes the dataset by renaming folders and files for better organization and efficiency. For instance, zipping the `NEU_Metal_Surface_Defects_Data/train` folder into `train_16.zip` produces the following structure:

<pre style="font-family: 'Courier New', monospace; font-size: 14px; line-height: 1.5; color: #333;">
Dataset/
├── 00000/
│   ├── img00000000.png
│   ├── img00000001.png
│   ├── ...
│   ├── img00000999.png
├── 00001/
│   ├── img00001000.png
│   ├── img00001001.png
│   ├── ...
│   ├── img00001999.png
├── ...
├── 0000N/
│   ├── img0000N000.png
│   ├── img0000N001.png
│   ├── ...
│   ├── img0000N999.png
├── dataset.json
├── label_map.json  
</pre>

#### Metadata Files:
1. **`dataset.json`**: This file maps images to their corresponding labels for TAO's StyleGAN-XL data loader. Example:  
    ```json
    {
        "labels": [
            ["00000/img00000000.png", 2],
            ["00000/img00000001.png", 4],
            ...
            ["00049/img00049999.png", 1]
        ]
    }
    ```
2. **`label_map.json`**: This file maps the original dataset's folder names to integer labels. Example:  
    ```json
    {
        "class_5": 0,
        "class_0": 1,
        "class_1": 2,
        "class_4": 3,
        "class_2": 4,
        "class_3": 5
    }
    ```

#### Label Mapping:  
The integer labels in `dataset.json` (e.g., `2`, `4`, `1`) differ from the original subfolder names. You can use `label_map.json` to interpret the mapping. For example:
- Label `2` corresponds to the original `class_1` folder.
- Label `4` corresponds to `class_2`.
- Label `1` corresponds to `class_0`.

#### Next Steps:  
We will use `dataset_convert` script to generate zipped versions of the dataset for all required resolutions (`16x16`, `32x32`, `64x64`, `128x128`, and `256x256`) from the original `NEU_Metal_Surface_Defects_Data/train`.

In [ ]:
# preprocess and generate train_16.zip
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_16.zip \
                  resolution=[16,16]

In [ ]:
# preprocess and generate train_32.zip
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_32.zip \
                  resolution=[32,32]

In [ ]:
# preprocess and generate train_64.zip
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_64.zip \
                  resolution=[64,64]

In [ ]:
# preprocess and generate train_128.zip
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_128.zip \
                  resolution=[128,128]

In [ ]:
# preprocess and generate train_256.zip
!tao model stylegan_xl dataset_convert \
                  -e $SPECS_DIR/dataset_convert.yaml \
                  source=$DATA_DIR/NEU_Metal_Surface_Defects_Data/train \
                  results_dir=$RESULTS_DIR/dataset_convert \
                  dest_file_name=train_256.zip \
                  resolution=[256,256]

### 2.3 Prepare required pretrained modules for training StyleGAN-XL

We will need 2 pretrained modules for training StyleGAN-XL: 
1. tf_efficientnet_lite0_embed.pth: Input embedding for input fed into the StyleGAN-XL
2. InceptionV3.pth: Evaluating FID score when training StyleGAN-XL

In [ ]:
! cd $HOST_DATA_DIR && \
    git clone https://github.com/autonomousvision/stylegan-xl.git && \
    cd stylegan-xl && wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/metrics/inception-2015-12-05.pkl

In [ ]:
import pickle
import sys
import torch
import os
sys.path.append(os.path.join(os.environ["HOST_DATA_DIR"], 'stylegan-xl'))
InceptionV3_file_path = os.path.join(os.environ["HOST_DATA_DIR"], 'stylegan-xl', "inception-2015-12-05.pkl")
tf_efficientnet_lite0_embed_file_path = os.path.join(os.environ["HOST_DATA_DIR"], 'stylegan-xl', "in_embeddings/tf_efficientnet_lite0.pkl")

# Try loading the checkpoint using pickle
with open(InceptionV3_file_path, 'rb') as f:
    InceptionV3 = pickle.load(f)
with open(tf_efficientnet_lite0_embed_file_path, 'rb') as f:
    tf_efficientnet_lite0_embed = pickle.load(f)

torch.save(InceptionV3.state_dict(), os.path.join(os.environ["HOST_DATA_DIR"], "InceptionV3.pth"))
torch.save(tf_efficientnet_lite0_embed['embed'].state_dict(), os.path.join(os.environ["HOST_DATA_DIR"], "tf_efficientnet_lite0_embed.pth"))

## 3. Run TAO train

### 3.0 Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training process. Please ensure you update the following settings to suit your environment:  

1. **`results_dir`**: Update this field if the default path is not suitable for your setup.  
2. **Dataset Paths**: Modify the `images_dir` under `train_dataset`, `validation_dataset`, and `test_dataset` to point to **your** dataset zip files as outlined in **Section 2.2**.

#### Progressive Training  
Since StyleGAN-XL utilizes a progressive training approach, we have prepared six specification files to accommodate this workflow:  
- `stylegan_xl_16.yaml`  
- `stylegan_xl_16_to32.yaml`  
- `stylegan_xl_16_to32_to64.yaml`  
- `stylegan_xl_16_to32_to64_to128.yaml`  
- `stylegan_xl_16_to32_to64_to128_to256.yaml`    

These files allow the model to progressively learn to generate images at increasing resolutions, ultimately achieving a final resolution of `256x256`.


### 3.1 Training the Stem (Base Low Resolution, e.g., 16x16) Model

In the specification file, the `superres` key is set to `False`, indicating that this is the base stem model. The `resolution` for the stem model is configured to `16`, matching the `img_resolution` of the dataset. The training dataset used is the `train_16.zip` file, which contains images at a resolution of 16x16.

In [ ]:
!tao model stylegan_xl train \
                  -e $SPECS_DIR/stylegan_xl_16.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16 \
                  model.input_embeddings_path=$DATA_DIR/tf_efficientnet_lite0_embed.pth \
                  model.stylegan.metrics.inception_fid_path=$DATA_DIR/InceptionV3.pth \
                  dataset.stylegan.train_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_16.zip \
                  dataset.stylegan.validation_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_16.zip \
                  dataset.stylegan.batch_gpu_size=8 \
                  dataset.batch_size=128

### 3.2 Training a Super-Resolution Model at 2x Resolution (32x32)

In the specification file, the `superres` key is set to `True`, and the `up_factor` is `[2]`, indicating a single upscaling step with a 2x magnification. The image dataset has an `img_resolution` of `32`, corresponding to the zipped dataset file `train_32.zip`. For this training phase, we use the latest checkpoint, `/results/stylegan_xl_16/train/styleganxl_model_latest.pth`, as the base model, which was trained at the previous resolution (16x16).

In [ ]:
!tao model stylegan_xl train \
                  -e $SPECS_DIR/stylegan_xl_16_to32.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32 \
                  model.input_embeddings_path=$DATA_DIR/tf_efficientnet_lite0_embed.pth \
                  model.stylegan.metrics.inception_fid_path=$DATA_DIR/InceptionV3.pth \
                  dataset.stylegan.train_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_32.zip \
                  dataset.stylegan.validation_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_32.zip \
                  dataset.stylegan.batch_gpu_size=8 \
                  dataset.batch_size=128

### 3.3 Training a Super-Resolution Model at 4x Resolution (64x64)

In the specification file, the `superres` key is set to `True`, and the `up_factor` is `[2, 2]`, indicating two upscaling steps, each with a 2x magnification. The image dataset has an `img_resolution` of `64`, corresponding to the zipped dataset file `train_64.zip`. For this training phase, we use the latest checkpoint, `/results/stylegan_xl_16_to32/train/styleganxl_model_latest.pth`, as the base model, which was trained at the previous resolution (32x32).

In [ ]:
!tao model stylegan_xl train \
                  -e $SPECS_DIR/stylegan_xl_16_to32_to64.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32_to64 \
                  model.input_embeddings_path=$DATA_DIR/tf_efficientnet_lite0_embed.pth \
                  model.stylegan.metrics.inception_fid_path=$DATA_DIR/InceptionV3.pth \
                  dataset.stylegan.train_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_64.zip \
                  dataset.stylegan.validation_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_64.zip \
                  dataset.stylegan.batch_gpu_size=8 \
                  dataset.batch_size=128

### 3.4 Training a Super-Resolution Model at 8x Resolution (128x128)

In the specification file, the `superres` key is set to `True`, and the `up_factor` is `[2, 2, 2]`, indicating three upscaling steps, each with a 2x magnification. The image dataset has an `img_resolution` of `128`, corresponding to the zipped dataset file `train_128.zip`. For this training phase, we use the latest checkpoint, `/results/stylegan_xl_16_to32_to64/train/styleganxl_model_latest.pth`, as the base model, which was trained at the previous resolution (64x64).

In [ ]:
!tao model stylegan_xl train \
                  -e $SPECS_DIR/stylegan_xl_16_to32_to64_to128.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128 \
                  model.input_embeddings_path=$DATA_DIR/tf_efficientnet_lite0_embed.pth \
                  model.stylegan.metrics.inception_fid_path=$DATA_DIR/InceptionV3.pth \
                  dataset.stylegan.train_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_128.zip \
                  dataset.stylegan.validation_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_128.zip \
                  dataset.stylegan.batch_gpu_size=8 \
                  dataset.batch_size=128

### 3.5 Training a Super-Resolution Model at 16x Resolution (256x256)

In the specification file, the `superres` key is set to `True`, and the `up_factor` is `[2, 2, 2, 2]`, indicating four upscaling steps, each with a 2x magnification. The image dataset has an `img_resolution` of `256`, corresponding to the zipped dataset file `train_256.zip`. For this training phase, we use the latest checkpoint, `/results/stylegan_xl_16_to32_to64_to128/train/styleganxl_model_latest.pth`, as the base model, which was trained at the previous resolution (128x128).

In [ ]:
!tao model stylegan_xl train \
                  -e $SPECS_DIR/stylegan_xl_16_to32_to64_to128_to256.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256 \
                  model.input_embeddings_path=$DATA_DIR/tf_efficientnet_lite0_embed.pth \
                  model.stylegan.metrics.inception_fid_path=$DATA_DIR/InceptionV3.pth \
                  dataset.stylegan.train_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_256.zip \
                  dataset.stylegan.validation_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_256.zip \
                  dataset.stylegan.batch_gpu_size=8 \
                  dataset.batch_size=128

## 4. Run TAO Evaluation <a class="anchor" id="head-3"></a>

In this section, we run the `stylegan_xl` evaluation script to assess the performance of the trained StyleGAN-XL model by comparing the FID scores between real and synthetic images. Assuming that we have trained the 256x256 model, we set the `checkpoint` path in the `evaluate` section to the location of the `$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256/train/styleganxl_model_latest.pth` checkpoint file.

In [ ]:
!tao model stylegan_xl evaluate \
                  -e $SPECS_DIR/stylegan_xl_16_to32_to64_to128_to256.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256 \
                  evaluate.checkpoint=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256/train/styleganxl_model_latest.pth \
                  dataset.stylegan.test_dataset.images_dir=$RESULTS_DIR/dataset_convert/train_256.zip \
                  dataset.batch_size=16

## 5. Run TAO Inference <a class="anchor" id="head-3"></a>

In this section, we run the `stylegan_xl` inference script to generate synthetic images using the trained StyleGAN-XL model. Assuming that we have trained the 256x256 model, we set the `checkpoint` path in the `inference` section to the location of the `$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256/train/styleganxl_model_latest.pth` checkpoint file.

This StyleGAN-XL model is trained as a conditional GAN that can generate images from one of the six classes in the `NEU_Metal_Surface_Defects_Data/train` dataset. In the `stylegan_xl_16_to32_to64_to128_to256.yaml` specification file, the `class_idx` is currently set to `0`, but you can override it to any value between `0` and `5` to generate images from different classes.

In [ ]:
!tao model stylegan_xl inference \
                  -e $SPECS_DIR/stylegan_xl_16_to32_to64_to128_to256.yaml \
                  results_dir=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256 \
                  inference.checkpoint=$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256/train/styleganxl_model_latest.pth \
                  dataset.stylegan.infer_dataset.start_seed=0 \
                  dataset.stylegan.infer_dataset.end_seed=50 \
                  inference.class_idx=0 \
                  dataset.batch_size=16

You can visualize the generated images from `$RESULTS_DIR` directory. The default path is `$RESULTS_DIR/stylegan_xl_16_to32_to64_to128_to256/inference/`

This notebook has come to an end.